<a href="https://colab.research.google.com/github/giustinod/nir-data/blob/main/Preparazione_dataset_dvoptic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importazioni librerie e definizione funzioni di utilità**

In [7]:
!pip3 install sklearn
!pip3 install spectral
!pip3 install pyod
!pip3 install cleanlab
!pip3 install scikeras[tensorflow]

## Import external libraries
import os
import sys
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

import tensorflow as tf

import scipy
from scipy import newaxis as nA
from scipy.signal import savgol_filter

from spectral import *
import spectral.io.envi as envi

from pyod.models.copod import COPOD

import multiprocessing
import time
from datetime import datetime
import concurrent.futures
from multiprocessing import Process

def snv(input_data):
    # Define a new array and populate it with the corrected data  
    # return (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
    output_data = np.zeros_like(input_data)
    for i in range(input_data.shape[0]):
      # if np.all(np.std(input_data[i,:]) != 0):
      # Apply correction
      output_data[i,:] = (input_data[i,:] - np.mean(input_data[i,:])) / (np.std(input_data[i,:]))
      # else:
      # output_data[i,:] = (input_data[i,:] - np.mean(input_data[i,:]))
    return output_data

def plot_matrix(matrixToPlot, title='', classList=[]):
    # colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]
    colors = ['b','c','r','g','w','k']
    cm = LinearSegmentedColormap.from_list('custom_RGB_cmap', colors, N=len(classList))
    norm = plt.Normalize(vmin=0, vmax=len(classList) - 1)
    handles = [plt.Rectangle((0, 0), 0, 0, color=cm(norm(i)), label=str(classList[i])) for i in range(len(classList))]
    plt.imshow(matrixToPlot, norm=norm, cmap=cm)
    plt.legend(handles=handles, title='Class', bbox_to_anchor=(1, 0), loc='lower right', bbox_transform=plt.gcf().transFigure)
    plt.title(title)
    plt.tight_layout()
    plt.savefig(title + '.png', dpi=300)
    plt.show()

def mlr(x, y, order):
    """Multiple linear regression fit of the columns of matrix x 
    (dependent variables) to constituent vector y (independent variables)
    
    order -     order of a smoothing polynomial, which can be included 
                in the set of independent variables. If order is
                not specified, no background will be included.
    b -         fit coeffs
    f -         fit result (m x 1 column vector)
    r -         residual   (m x 1 column vector)
    """
    if order > 0:
        s = scipy.ones((len(y),1))
        for j in range(order):
            s = scipy.concatenate((s,(scipy.arange(0,1+(1.0/(len(y)-1)),1.0/(len(y)-1))**j)[:x.shape[0],nA]), 1)
        X = scipy.concatenate((x, s), 1)
    else:
        X = x
    
    #calc fit b=fit coefficients
    b = scipy.dot(scipy.dot(scipy.linalg.pinv(scipy.dot(scipy.transpose(X),X)),scipy.transpose(X)),y)
    f = scipy.dot(X,b)
    r = y - f

    return b,f,r

def emsc(myarray, order = 1, fit = None):
    """Extended multiplicative scatter correction (Ref H. Martens)
    myarray -   spectral data for background correction
    order -     order of polynomial
    fit -       if None then use average spectrum, otherwise provide a spectrum
                as a column vector to which all others fitted
    corr -      EMSC corrected data
    mx -        fitting spectrum
    """
    
    #choose fitting vector
    if fit:
        mx = fit
    else:
        mx = scipy.mean(myarray, axis=0)[:,nA]

    #do fitting
    corr = scipy.zeros(myarray.shape)
    for i in range(len(myarray)):
        if np.count_nonzero(myarray[i,:]) > 0:
            b,f,r = mlr(mx, myarray[i,:][:,nA], order)
            corr[i,:] = scipy.reshape((r/b[0,0]) + mx, (corr.shape[1],))

    assert not np.any(np.isnan(corr))
    return corr

def data_augmentation(scan):
  ## Standardize on columns
  if np.count_nonzero(scan) > 0:
      svn_scan = snv(scan)
      msc_scan = emsc(scan)
      scan1 = np.concatenate((scan, svn_scan, msc_scan,\
                  savgol_filter(scan, w, polyorder = p, deriv=1),\
                  savgol_filter(svn_scan, w, polyorder = p, deriv=1),\
                  savgol_filter(msc_scan, w, polyorder = p, deriv=1)), axis = 1)
  else:
      scan1 = np.zeros((scan.shape[0], scan.shape[1] * 6), dtype=np.float32)

  print('{} - Data augmentation chunk: {} -> {}'.format(datetime.now(), scan.shape, scan1.shape))
  return scan1

# Check speculari
def outliers_speculari(input_data):

  ''' Restituisce gli indici degli spettri che hanno caratteristiche di speculari'''
  # per lo spettro IR [1050,1650]nm assumiamo:
  IR_delta_max = 0.5
  IR_lambda_max = 1.2
  # per lo spettro VNIR [[400, 600], [800, 900]]nm assumiamo:
  VNIR_lambda_max = 1.2

  offset = int((1010 - 400) / 10)
  offset2 = int((600 - 400) / 10)
  offset3 = int((800 - 400) / 10)
  offset4 = int((900 - 400) / 10)
  speculari_VNIR = np.where((np.max(input_data[:, :offset2], axis = 1) > VNIR_lambda_max) | (np.max(input_data[:, offset3:offset4], axis = 1) > VNIR_lambda_max)) 
  speculari_IR = np.where((np.max(input_data[:, offset + 1:], axis = 1) > IR_lambda_max) | ((np.max(input_data[:, offset + 1:], axis = 1) - np.min(input_data[:, offset + 1:], axis = 1)) > IR_delta_max))

  # print('Speculari IR: ', speculari_IR)
  # print('Speculari VNIR: ', speculari_VNIR)
  speculari = np.unique(np.concatenate((speculari_IR[0], speculari_VNIR[0]), axis=0))
  # print('Speculari: ', speculari)
  return speculari

def conv(cl):
    if cl == 'Carta':
        return 0
    if cl == 'VetroCarta':
        return 1
    if cl == 'Ceramica':
        return 2
    if cl == 'Opalino':
        return 3
    if cl == 'Plastica':
        return 4
    if cl == 'Vetro':
        return 5
    if cl == 'VetroCeramica':
        return 6
    if cl == 'BackGround':
        return 7
    return 9   # default case if x is not found

selected_classes = (0, 1, 2, 3, 4, 5, 6, 7)
n_classes = len(selected_classes)

## Settings for the smooth derivatives using a Savitsky-Golay filter
w = 7 ## Sav.Gol window size
p = 2 ## Sav.Gol polynomial degree

n_features = 182

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found GPU at: /device:GPU:0


**Dalle immagini HDR estraggo i CSV eliminando i pixel a zero e gli intorni (+ o - 3 wl)**

In [18]:
# 1. Authenticate to google drive and mount drive.
from google.colab import drive
drive.mount('/content/drive')

root = '/content/drive/MyDrive/Freelancer/GraiNit/dvoptic/merge/'

# mpy = COPOD()

# cfr. https://stackoverflow.com/questions/9132114/indexing-numpy-array-neighbours-efficiently
def neighbors(x):
    return np.array([x-3, x-2, x-1, x, x+1, x+2, x+3])

for path, subdirs, files in os.walk(root):
    # print('Path: ', path)
    for name in files:
        if name.endswith('hdr'):
            print('{} - Opening {}'.format(datetime.now(), os.path.join(path, name)))
            MyImg = envi.open(os.path.join(path, name))
            np_img = np.array(MyImg.load())
            # print('{} - Image shape: {}'.format(datetime.now(), np_img.shape))
            # print('{} - Image rows*cols: {}*{}'.format(datetime.now(), MyImg.nrows, MyImg.ncols))
            X = np.resize(np_img, (np_img.shape[0] * np_img.shape[1], np_img.shape[2]))
            # print('{} - Conv. to Numpy array {}'.format(datetime.now(), X.shape))
            # spec_outliers = outliers_speculari(X)
            # print('{} - Outliers Speculari # {}'.format(datetime.now(), np.unique(spec_outliers).shape))
            # fitted = mpy.fit(X)
            # c_outliers = mpy.predict(X)
            # copod_outliers = np.where(c_outliers >= 1)
            # print('{} - Outliers COPOD # {}'.format(datetime.now(), copod_outliers[0].shape))
            # mask_outliers = spec_outliers # np.unique(np.concatenate((spec_outliers, copod_outliers[0])))
            mask_all_zeroes = np.where(~X.any(axis=1))[0]
            nb_all_zeroes = np.unique(np.concatenate([neighbors(i) for i in mask_all_zeroes]))
            # print('{} - Rows having all zeroes # {}'.format(datetime.now(), mask_all_zeroes))
            # print('{} - Rows having all zeroes in the neighborhood # {} - {}'.format(datetime.now(), nb_all_zeroes, nb_all_zeroes.shape))
            nb_all_zeroes = np.delete(nb_all_zeroes, [0, 1, 2, nb_all_zeroes.shape[0]-3, nb_all_zeroes.shape[0]-2, nb_all_zeroes.shape[0]-1], axis = 0)
            X = np.delete(X, nb_all_zeroes, axis = 0)
            # print('{} - Rows having all zeroes in the neighborhood # {} - {}'.format(datetime.now(), nb_all_zeroes, nb_all_zeroes.shape))
            print('{} - Result size # {}'.format(datetime.now(), X.shape))
            cols = np.concatenate([np.arange(400, 1010, 10), np.arange(1050, 1655, 5)])
            df = pd.DataFrame(X, columns = cols)
            date_ = name.split('_')[0]
            class_ = name.split('_')[1]
            class_col = np.full(X.shape[0], class_)
            csv = df.insert(0, "Class", class_col)
            filename = name.replace('hdr', 'csv')
            print('{} - csv # {}'.format(datetime.now(), filename))
            df.to_csv('/content/drive/MyDrive/Freelancer/GraiNit/dvoptic/training/' + filename, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2022-09-22 09:19:07.995205 - Opening /content/drive/MyDrive/Freelancer/GraiNit/dvoptic/merge/23062022_Ceramica_2Merge.hdr
2022-09-22 09:19:09.773284 - Result size # (46481, 182)
2022-09-22 09:19:09.778704 - csv # 23062022_Ceramica_2Merge.csv
2022-09-22 09:19:16.666024 - Opening /content/drive/MyDrive/Freelancer/GraiNit/dvoptic/merge/04072022_Ceramica_Sassi0Merge.hdr
2022-09-22 09:19:18.501656 - Result size # (40295, 182)
2022-09-22 09:19:18.506717 - csv # 04072022_Ceramica_Sassi0Merge.csv
2022-09-22 09:19:24.495302 - Opening /content/drive/MyDrive/Freelancer/GraiNit/dvoptic/merge/04072022_Ceramica_Scure0Merge.hdr
2022-09-22 09:19:26.498311 - Result size # (45427, 182)
2022-09-22 09:19:26.503398 - csv # 04072022_Ceramica_Scure0Merge.csv
2022-09-22 09:19:33.598905 - Opening /content/drive/MyDrive/Freelancer/GraiNit/dvoptic/merge/04072022_Ceramica_Latterizio1Mer

**Predisposizione dataset per training, validazione e test**

In [42]:
## Import external libraries

roots = ['/content/drive/MyDrive/Freelancer/GraiNit/dvoptic/training/',]

columns = np.array(['Class'])

df_train = pd.DataFrame(data=None, columns=columns)
df_test = pd.DataFrame(data=None, columns=columns)
df_val = pd.DataFrame(data=None, columns=columns)

perc_train = 0.55
perc_test = 0.2
perc_val = 0.25

# dataset ricevuti dal 1 giugno con aggiunta BG 25 maggio
perc_class = [0.5, 0.25, 0.25, 1, 1, 0.5, 0.51, 0.13]

for r in roots:
    for path, subdirs, files in os.walk(r):
        for name in files:
            if name.endswith('csv'):
                df = pd.read_csv(path + '/' + name, header = 0)
                m = re.search('04072022_(.+?)_(.+?).csv', name)
                class_ = conv(m.group(1))
                df.replace(m.group(1), class_, inplace = True)
                df.dropna(how='any', inplace = True)
                df.rename(columns={'Unnamed: 0':'Class'}, inplace = True)
                cols = np.concatenate([['Class'], np.concatenate([np.arange(400, 1010, 10), np.arange(1050, 1655, 5)])])
                df.columns = cols
                n_rows = df.shape[0]
                n_rows_train = round(n_rows * perc_train * perc_class[class_])
                n_rows_test = round(n_rows * perc_test * perc_class[class_])
                n_rows_val = round(n_rows * perc_val * perc_class[class_])
                print('{} ({}): {}, {}'.format(m.group(1), name, str(n_rows), str(n_rows_train)))
                df_train = pd.concat([df_train, df.loc[0:n_rows_train,:]], ignore_index = True)
                df_test = pd.concat([df_test, df.loc[n_rows_train+1:n_rows_train+n_rows_test,:]], ignore_index = True)
                df_val = pd.concat([df_val, df.loc[n_rows_train+n_rows_test+1:n_rows_train+n_rows_test+n_rows_val,:]], ignore_index = True)

# removing duplicates
df_train.drop_duplicates(inplace = True)
df_test.drop_duplicates(inplace = True)
df_val.drop_duplicates(inplace = True)

print(df_train.groupby(['Class']).count().iloc[:, :1])
df_train.to_csv('/content/drive/MyDrive/Freelancer/GraiNit/dvoptic/' + 'DataSetTraining11.csv', index = False)
print(df_test.groupby(['Class']).count().iloc[:, :1])
df_test.to_csv('/content/drive/MyDrive/Freelancer/GraiNit/dvoptic/' + 'DataSetTest11.csv', index = False)
print(df_val.groupby(['Class']).count().iloc[:, :1])
df_val.to_csv('/content/drive/MyDrive/Freelancer/GraiNit/dvoptic/' + 'DataSetValidation11.csv', index = False)


Ceramica (04072022_Ceramica_2Merge.csv): 46481, 6391
Ceramica (04072022_Ceramica_Sassi0Merge.csv): 40295, 5541
Ceramica (04072022_Ceramica_Scure0Merge.csv): 45427, 6246
Ceramica (04072022_Ceramica_Latterizio1Merge.csv): 20640, 2838
Ceramica (04072022_Ceramica_Colore1Merge.csv): 65016, 8940
Ceramica (04072022_Ceramica_Bianca2Merge.csv): 62592, 8606
Vetro (04072022_Vetro_5Merge.csv): 54889, 15094
Vetro (04072022_Vetro_1Merge.csv): 61725, 16974
Plastica (04072022_Plastica_Buona2Merge.csv): 6022, 3312
Plastica (04072022_Plastica_Buona1Merge.csv): 30207, 16614
Opalino (04072022_Opalino_1Merge.csv): 10311, 5671
Opalino (04072022_Opalino_0Merge.csv): 34499, 18974
BackGround (04072022_BackGround_All.csv): 520266, 37199
Carta (04072022_Carta_Organico.csv): 42612, 11718
VetroCeramica (04072022_VetroCeramica_All.csv): 125934, 35324
VetroCarta (04072022_VetroCarta_All.csv): 261280, 35926
Carta (04072022_Carta_Cartone.csv): 71459, 19651
Ceramica (04072022_Ceramica_2Merge (1).csv): 46481, 6391
     